<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
# get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-08-22 11:39:01--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  35.5MB/s    in 7.8s    

2020-08-22 11:39:09 (32.1 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
# and upzip it
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [3]:
# import tensorflow
import tensorflow as tf

In [4]:
# global constants
num_classes = 2
image_resize = 224

batch_size_training = 100
batch_size_validation = 100

In [5]:
# ImageDataGenerator instance
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

# Training Data Generator
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

# Validation Data Generator
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [6]:
# VGG-16 pre-trained model
vgg_model = tf.keras.models.Sequential(name="vgg-16")
vgg_model.add(tf.keras.applications.VGG16(include_top=False, pooling='avg', weights='imagenet'))

# custom dense layer
vgg_model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

58892288/58889256 [==============================] - 0s 0us/step


In [7]:
# make vgg-16 layers non-trainable
print("Layers in Model: {}".format(vgg_model.layers))
vgg_model.layers[0].trainable = False

Layers in Model: [<tensorflow.python.keras.engine.functional.Functional object at 0x7f8a78784908>, <tensorflow.python.keras.layers.core.Dense object at 0x7f8a7878b6d8>]


In [8]:
print("vgg model summary:\n")
vgg_model.summary()

vgg model summary:

Model: "vgg-16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [9]:
# variables for model.fit()
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

print("\nsteps_per_epoch_training: {} and steps_per_epoch_validation: {}\n".format(steps_per_epoch_training, steps_per_epoch_validation))

# Compile VGG Model
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


steps_per_epoch_training: 301 and steps_per_epoch_validation: 96



In [10]:
# Fit VGG Model
print("\nvgg model:\n")
vgg_history = vgg_model.fit(train_generator,
                          steps_per_epoch=steps_per_epoch_training,
                          epochs=num_epochs,
                          validation_data=validation_generator,
                          validation_steps=steps_per_epoch_validation,
                          verbose=1)


vgg model:

Epoch 1/2
301/301 [==============================] - 102s 339ms/step - loss: 0.1508 - accuracy: 0.9459 - val_loss: 0.0376 - val_accuracy: 0.9932
Epoch 2/2
301/301 [==============================] - 103s 341ms/step - loss: 0.0288 - accuracy: 0.9940 - val_loss: 0.0211 - val_accuracy: 0.9954


In [11]:
# Save Model
vgg_model.save("vgg-16.h5")

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [ ]:
# Load ResNet50 Model
resnet_model = tf.keras.models.load_model("resnet-50.h5")

In [13]:
# Test Data Generator
test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator()

test_generator = test_data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle=False)

Found 500 images belonging to 2 classes.


In [14]:
# Evaluate VGG Model
vgg_loss, vgg_acc = vgg_model.evaluate(test_generator, steps=len(test_generator), verbose=1)

16/16 [==============================] - 2s 95ms/step - loss: 0.1809 - accuracy: 0.9220


In [15]:
# Evaluate ResNet Model
resnet_loss, resnet_acc = resnet_model.evaluate(test_generator, steps=len(test_generator), verbose=1)

16/16 [==============================] - 1s 74ms/step - loss: 0.0817 - accuracy: 0.9700


In [16]:
# Print Results
print("\nVGG Model Loss: {} and Accuracy: {}\n".format(vgg_loss, vgg_acc))
print("\nResNet Model Loss: {} and Accuracy: {}\n".format(resnet_loss, resnet_acc))


VGG Model Loss: 0.1809215098619461 and Accuracy: 0.921999990940094


ResNet Model Loss: 0.08167499303817749 and Accuracy: 0.9700000286102295



<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [17]:
# Predict Results with VGG
vgg_results = vgg_model.predict(test_generator)

In [18]:
# Predict Results with ResNet
resnet_results = resnet_model.predict(test_generator)

In [26]:
# import numpy
import numpy as np

# Print first 5 predictions : VGG
v_results = vgg_results[:5]

print("VGG Result: \n")
for result in v_results:
  res = np.argmax(result)
  if res == 0:
    print("Negative")
  else:
    print("Positive")

VGG Result: 

Negative
Negative
Negative
Negative
Negative


In [27]:
# Print first 5 predictions : ResNet
r_results = resnet_results[:5]

print("ResNet Result: \n")
for result in r_results:
  res = np.argmax(result)
  if res == 0:
    print("Negative")
  else:
    print("Positive")

ResNet Result: 

Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).